In [ ]:
Fetch posts and comments, for posts that have comments. 

In [4]:
# Import necessary libraries
import sqlite3
import json
import os
from datetime import datetime

# Paths for the database and output file
db_path = '/Users/buddy/Desktop/School/effective-adventure/effective-adventure/db/WGU_Reddit.db'
output_path = '/Users/buddy/Desktop/School/effective-adventure/effective-adventure/data/most_recent_post_with_comments.json'

# Function to fetch the most recent post with all its comments
def fetch_most_recent_post_with_comments():
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Query to fetch the most recent post with comments
    cursor.execute("""
    SELECT * FROM posts
    WHERE num_comments > 10
    ORDER BY created_utc DESC
    LIMIT 10
    """)
    post = cursor.fetchone()
    post_columns = [column[0] for column in cursor.description]
    post_data = dict(zip(post_columns, post)) if post else None
    
    # Check if there is a post with comments
    if post_data:
        post_id = post_data['post_id']
        
        # Query to fetch all comments for the most recent post
        cursor.execute("""
        SELECT * FROM comments
        WHERE post_id = ?
        """, (post_id,))
        comments = cursor.fetchall()
        comment_columns = [column[0] for column in cursor.description]
        comments_data = [dict(zip(comment_columns, row)) for row in comments]
        
        # Nest comments under the post
        post_data['comments'] = comments_data
    
        # Save the nested structure to JSON
        with open(output_path, 'w') as f:
            json.dump(post_data, f, indent=4, default=str)
    
    # Close the connection
    conn.close()

# Run the function to fetch and save data
fetch_most_recent_post_with_comments()

print("Most recent post with its comments has been successfully saved.")

Most recent post with its comments has been successfully saved.


## get high ratio comments

In [8]:
import sqlite3
import pandas as pd

# Path to the database
db_path = '/Users/buddy/Desktop/School/effective-adventure/effective-adventure/db/WGU_Reddit.db'

# Connect to the database
conn = sqlite3.connect(db_path)

# Query to fetch and sort comments by ratio with parent post details
query = '''
    SELECT 
        posts.title AS post_title,
        posts.selftext AS post_selftext,
        posts.score AS post_score,
        comments.content AS comment_content,
        comments.upvotes AS comment_score,
        comments.ratio AS comment_ratio
    FROM comments
    JOIN posts ON comments.post_id = posts.post_id
    ORDER BY comments.ratio DESC
'''

# Load the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Truncate 'post_title' and 'post_selftext' to 30 characters
df['post_title'] = df['post_title'].str.slice(0, 30)
df['post_selftext'] = df['post_selftext'].str.slice(0, 30)

# Display the DataFrame sorted by ratio, showing the top results
df[['post_title', 'post_selftext', 'post_score', 'comment_content', 'comment_score', 'comment_ratio']].head(100)

,post_title,post_selftext,post_score,comment_content,comment_score,comment_ratio
0,Switch majors,I’m considering switching from,1,You can still apply for those jobs with a soft...,20,20.0
1,DSA II (C950) - Is copying cod,I just started my DSA II proje,1,My opinion: the best way to learn is to take a...,19,19.0
2,Background check new and exist,I'm considering applying for a,1,Agreed. I never like to discourage people from...,19,19.0
3,Class Difficulty Ranking,I am starting my first term fo,1,Google each class and see what people say. Be...,19,19.0
4,truth behind accelerating in e,You've probably heard it befo,1,This post kind of confuses me. To begin with ...,18,18.0
...,...,...,...,...,...,...
95,My WGU Experience,I thought I'd share my journey,1,Just wanted to say reading this gave me hope. ...,6,6.0
96,My WGU Experience,I thought I'd share my journey,1,"Yeah, that one was a lot and I was in a simila...",6,6.0
97,Switch majors,I’m considering switching from,1,I have spoken with a few software engineers an...,6,6.0
98,Switch majors,I’m considering switching from,1,Talk to your mentor,6,6.0


In [9]:
# Filter the DataFrame for comments with post_score greater than 1
filtered_df = df[df['post_score'] > 1]

# Display the filtered DataFrame
filtered_df[['post_title', 'post_selftext', 'post_score', 'comment_content', 'comment_score', 'comment_ratio']].head(10)

,post_title,post_selftext,post_score,comment_content,comment_score,comment_ratio
14,Failed C213 twice,I am seriously lost… I did 10x,2,Sounds like the third time will do the trick! ...,25,12.500000
15,Wondering how long this degree,How long do you guys think it,2,"Well, since we know you as a person so well, I...",24,12.000000
19,Got a job application rejectio,Applied on July 18 to their ap,3,Do you think other students/alumni applied doi...,35,11.666667
33,Can you be for real grammarly?,,2,">this comma is unnecessary, get rid of it\n\n>...",19,9.500000
39,D276 web development PA score.,I have no experience with web,4,This is foundational knowledge for a lot of jo...,34,8.500000
42,Friends?,Wgu marketing program? Anyone,2,You’ve gotta market yourself better than that.,16,8.000000
56,D276 web development PA score.,I have no experience with web,4,Well one tip would be learning how to take scr...,31,7.750000
57,Wondering how long this degree,How long do you guys think it,2,Just about as much effort as the question - it...,15,7.500000
58,How difficult are the business,I am debating whether to do th,2,[deleted],15,7.500000
59,Study.com while attending WGU?,Can I still bring in study.com,2,"No, my understanding is you have to transfer t...",15,7.500000


In [10]:
import sqlite3
import pandas as pd

# Set the path to your database
db_path = '/Users/buddy/Desktop/School/effective-adventure/effective-adventure/db/WGU_Reddit.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Query to fetch all data from the comments_new table
query = "SELECT * FROM comments_new"

# Load the data into a pandas DataFrame
comments_df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the count of each column
column_counts = comments_df.count()
print(column_counts)

id             90821
comment_id     90821
post_id        90821
parent_id          1
author         86876
content        90821
upvotes        90821
created_utc    90821
ratio          87535
dtype: int64


In [19]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
db_path = '/Users/buddy/Desktop/School/effective-adventure/effective-adventure/db/WGU_Reddit.db'
conn = sqlite3.connect(db_path)

# Query to fetch the top 10 post IDs and number of comments from comments_new
query = '''
SELECT post_id, 
       title, 
       num_comments
FROM posts
ORDER BY num_comments DESC
LIMIT 10
'''

# Execute the query and load results into a DataFrame
top_posts_df = pd.read_sql_query(query, conn)

# Display the results
print(top_posts_df)

# Close the database connection
conn.close()

   post_id                                              title  num_comments
0  1ggt6p0                                 NOV 1st CHECK IN 📋           299
1  1gia0jv  If you can get a BS in 6 months and for $4k, w...           283
2  1g8e2xv                                   Proctor Snooping           242
3  1ge45ck              WGU Is Harder Than A Brick and Mortar           202
4  1g2h8iv  My YouTube Channel Just Blew Up! Should I Paus...           194
5  1g1prh3                  Any younger folks attending WGU?            180
6  1g943tk                        Guardian Browser scares me            174
7  1g8720f  What’s something you wish you knew before star...           168
8  1esu2nj  WGU Annual All-Hands Megathread (Annual All Ha...           155
9  1gf34d5                                       I failed WGU           154


## show new comments


In [23]:
import sqlite3

# Connect to the SQLite database
db_path = '/Users/buddy/Desktop/School/effective-adventure/effective-adventure/db/WGU_Reddit.db'
connection = sqlite3.connect(db_path)
cursor = connection.cursor()

# Function to recursively print comments with indentation
def print_comments(comment_id, indent=0):
    # Fetch comments with the specified parent_id
    cursor.execute("SELECT * FROM comments_new WHERE parent_id = ?", (comment_id,))
    comments = cursor.fetchall()
    
    if not comments:
        return  # No comments to display for this parent_id

    for comment in comments:
        # Print the current comment with indentation
        print('\t' * indent + f"Comment ID: {comment[1]} | Author: {comment[4]} | Content: {comment[5]} | Upvotes: {comment[6]}")
        
        # Recursively print replies (children) to this comment
        print_comments(comment[1], indent + 1)

# Fetch and print all top-level comments
cursor.execute("SELECT * FROM comments_new WHERE parent_id IS NULL")
top_level_comments = cursor.fetchall()

if not top_level_comments:
    print("No top-level comments found.")
else:
    for comment in top_level_comments:
        print('\t' * 0 + f"Top-Level Comment ID: {comment[1]} | Author: {comment[4]} | Content: {comment[5]} | Upvotes: {comment[6]}")
        print_comments(comment[1])  # Print child comments for each top-level comment

# Close the database connection
connection.close()

No top-level comments found.


In [26]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
db_path = '/Users/buddy/Desktop/School/effective-adventure/effective-adventure/db/WGU_Reddit.db'
connection = sqlite3.connect(db_path)

# Fetch all data from the comments_new table
query = "SELECT * FROM comments_new"
df = pd.read_sql_query(query, connection)

# Specify the output CSV file path
csv_file_path = '/Users/buddy/Desktop/School/effective-adventure/effective-adventure/data/comments_new.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

# Close the database connection
connection.close()

print(f"Data exported to {csv_file_path}")

Data exported to /Users/buddy/Desktop/School/effective-adventure/effective-adventure/data/comments_new.csv
